In [310]:
import pandas as pd

In [311]:
df = pd.read_csv("../crawler/data/list_house.csv")
len(df)

2000

In [312]:
def check(df):
    l = []
    columns = df.columns
    for col in columns:
        dtypes = df[col].dtypes
        nunique = df[col].nunique()
        sum_null = df[col].isnull().sum()
        l.append([col, dtypes, nunique, sum_null])
    df_check = pd.DataFrame(l)
    df_check.columns = ['Column','Dtypes','Nunique','Sum_null']
    return df_check 
check(df)

,Column,Dtypes,Nunique,Sum_null
0,Unnamed: 0,int64,2000,0
1,Link,object,1999,0
2,Title,object,1965,23
3,Estate_type,object,9,423
4,Province,object,2,423
5,District,object,24,427
6,Ward,object,196,427
7,Price,object,633,32
8,Square,object,264,49
9,Post_date,object,28,2


In [313]:
df.dropna(subset=["Price", "Square"], inplace=True)

In [314]:
outliers = df[(df['Numb_bedroom'] > 1000) | (df['Numb_toilet'] > 1000) | (df['Numb_floor'] > 1000)]
len(outliers)

1

In [315]:
df = df[(df['Numb_bedroom'] < 100) & (df['Numb_toilet'] < 100) & (df['Numb_floor'] < 100)]

In [316]:
df.shape

(1354, 14)

In [317]:
for column in ['Estate_type', 'District', 'Ward']:
    df.loc[:, column] = df[column].fillna("Unknown")

df.loc[:, 'Province'] = df['Province'].fillna("Hà Nội")

for column in ['Numb_bedroom', 'Numb_toilet', 'Numb_floor']:
    df.loc[:, column] = df[column].fillna(df[column].mean())

In [318]:
df.loc[:, 'Square'] = df['Square'].str.replace(' m2', '', regex=False)

In [319]:
df['Square'] = pd.to_numeric(df['Square'], errors='coerce')
df = df.dropna(subset=['Square'])
print(df['Square'].dtype)

float64


In [320]:
df['Square'].head()

0    54.0
1    33.0
3    55.0
4    99.0
9    41.0
Name: Square, dtype: float64

In [321]:
import numpy as np

def convert_price(price_str):
    price_str = price_str.replace(',', '').strip()
    if 'tỷ' in price_str:
        return float(price_str.replace('tỷ', '').strip()) * 1e9
    elif 'triệu' in price_str:
        return float(price_str.replace('triệu', '').strip()) * 1e6
    else:
        try:
            return float(price_str)
        except ValueError:
            return np.nan

In [322]:
df['Price'] = df['Price'].apply(convert_price)
df = df.dropna(subset=['Price'])
print(df['Price'].head())

0    4.860000e+09
1    5.000000e+09
3    2.550000e+09
4    4.750000e+09
9    1.690000e+10
Name: Price, dtype: float64


In [323]:
check(df)

,Column,Dtypes,Nunique,Sum_null
0,Unnamed: 0,int64,1342,0
1,Link,object,1342,0
2,Title,object,1328,8
3,Estate_type,object,8,0
4,Province,object,1,0
5,District,object,18,0
6,Ward,object,151,0
7,Price,float64,428,0
8,Square,float64,181,0
9,Post_date,object,25,0


In [324]:
df = df[df['Price'] != 0]
len(df)

1299

In [325]:
df.to_csv("data_ETLed.csv")